In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from imutils import paths

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense , Flatten , AveragePooling2D , Dropout , Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

from tensorflow.keras.preprocessing.image import ImageDataGenerator , img_to_array , load_img
from tensorflow.keras.utils import to_categorical

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split



In [2]:
dataset = r"D:\ML\maskdetect\dataset"
imagePaths = list(paths.list_images(dataset))

In [3]:
data =[]
labels =[]

for i in imagePaths:
    label = i.split(os.path.sep)[-2]
    labels.append(label)
    image = load_img(i, target_size=(224 , 224))
    image = img_to_array(image)
    image = preprocess_input(image)
    data.append(image)

C:\Users\Avneesh\anaconda3\lib\site-packages\PIL\Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [4]:
data = np.array(data , dtype="float32")

In [5]:
labels = np.array(labels)

In [6]:
labels

array(['without_mask', 'without_mask', 'without_mask', ..., 'with_mask',
       'with_mask', 'with_mask'], dtype='<U12')

In [7]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [8]:
X_train , X_test , y_train , y_test = train_test_split(data , labels , test_size = 0.20 , random_state = 10 , stratify = labels)

In [9]:
aug = ImageDataGenerator(rotation_range=20 , zoom_range=0.15 , width_shift_range=0.2 , height_shift_range=0.2 , shear_range=0.15 , horizontal_flip=True , vertical_flip=True , fill_mode='nearest')

In [10]:
baseModel = MobileNetV2(weights = 'imagenet' , include_top = False , input_tensor = Input(shape=(224,224,3)))

9412608/9406464 [==============================] - 3s 0us/step


In [11]:
baseModel.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________

In [12]:
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7,7))(headModel)
headModel = Flatten(name = 'Flatten')(headModel)
headModel = Dense(128 , activation='relu')(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2 , activation = 'softmax')(headModel)

model = Model(inputs = baseModel.input , outputs = headModel)

In [13]:
for layer in baseModel.layers:
    layer.trainable = False

In [14]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________________

In [15]:
learning_rate = 0.0001
epochs = 20
BS = 12

opt = Adam(lr = learning_rate , decay=learning_rate/epochs)
model.compile(loss = 'binary_crossentropy' , optimizer=opt , metrics=['accuracy'])

model.fit(
    aug.flow(X_train , y_train , batch_size=BS),
    steps_per_epoch=len(X_train)//BS,
    validation_data=(X_test , y_test),
    validation_steps=len(X_test)//BS,
    epochs = epochs
)

model.save(r"D:\ML\maskdetect\my_model_2.model")

Epoch 1/20
255/255 [==============================] - 86s 339ms/step - loss: 0.3780 - accuracy: 0.8333 - val_loss: 0.0961 - val_accuracy: 0.9765
Epoch 2/20
255/255 [==============================] - 71s 279ms/step - loss: 0.1665 - accuracy: 0.9358
Epoch 3/20
255/255 [==============================] - 71s 280ms/step - loss: 0.1172 - accuracy: 0.9581
Epoch 4/20
255/255 [==============================] - 78s 307ms/step - loss: 0.1024 - accuracy: 0.9640
Epoch 5/20
255/255 [==============================] - 73s 288ms/step - loss: 0.0836 - accuracy: 0.9695
Epoch 6/20
255/255 [==============================] - 72s 282ms/step - loss: 0.0773 - accuracy: 0.9751
Epoch 7/20
255/255 [==============================] - 72s 282ms/step - loss: 0.0718 - accuracy: 0.9712
Epoch 8/20
255/255 [==============================] - 72s 282ms/step - loss: 0.0644 - accuracy: 0.9758
Epoch 9/20
255/255 [==============================] - 72s 281ms/step - loss: 0.0637 - accuracy: 0.9758
Epoch 10/20
255/255 [==========

In [16]:
from sklearn.metrics import classification_report

predict = model.predict(X_test , batch_size=BS)
predict = np.argmax(predict , axis=1)
print(classification_report(y_test.argmax(axis=1) , predict , target_names=lb.classes_))

              precision    recall  f1-score   support

   with_mask       0.99      0.99      0.99       383
without_mask       0.99      0.99      0.99       384

    accuracy                           0.99       767
   macro avg       0.99      0.99      0.99       767
weighted avg       0.99      0.99      0.99       767



In [20]:
pred = model.predict(X_test , batch_siz

array([[9.9991679e-01, 8.3166502e-05],
       [2.3749115e-05, 9.9997628e-01],
       [9.9999952e-01, 4.3987657e-07],
       ...,
       [9.6717256e-04, 9.9903286e-01],
       [2.0014243e-04, 9.9979991e-01],
       [2.6885904e-02, 9.7311407e-01]], dtype=float32)

In [26]:
y_test[:,1]

array([0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 1., 1.,
       0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1., 0., 0.,
       0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 0., 1., 1.,
       1., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0.,
       0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0.,
       1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0.,
       1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1.,
       1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 1., 1.,
       1., 0., 0., 0., 1., 0., 1., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0.,
       1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 0., 1., 1.,
       0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 1., 0., 1., 0.,
       1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 1.,
       0., 0., 1., 1., 0.

In [27]:
y_test[:,0]

array([1., 0., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1., 0., 1., 0., 0., 0.,
       1., 1., 0., 0., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1.,
       1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0.,
       0., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 1., 1., 1.,
       1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1., 0., 1., 1.,
       0., 1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1.,
       0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 0., 1., 0., 0.,
       0., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1.,
       0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0.,
       1., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1.,
       0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0.,
       1., 1., 0., 0., 1.